In [ ]:
import pandas as pd
import time
import pickle
import glob
import datetime
import numpy as np
import csv
import re
from time import strftime
import os
import statistics
from statistics import mode
pd.set_option('display.max_colwidth', None)

In [ ]:
df_cluster_tweet =  pd.read_csv("Latest_Tweet_Fake_Spreader_Tweets.csv", lineterminator='\n', dtype=str)
df_account = pd.read_csv('All_Fake_without_debunkers_Updated.csv')

In [ ]:
df_alt_report = pd.read_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/ML Model Data/AltNews/AltNews_Report_Cluster_Model_data_Updated.csv')

In [ ]:
df_cluster_classification = pd.read_csv('Cluster_Prediction.csv')
df_cluster_classification.Tweet_ID = df_cluster_classification.Tweet_ID.astype('str')

In [ ]:
df_cluster_tweet_mod = pd.merge(df_cluster_classification[['Cluster_ID', 'Tweet_ID', 'Tweet_Link', 'Prediction2']],df_cluster_tweet)

In [ ]:
df_cluster_tweet_mod = df_cluster_tweet_mod.drop(columns = ['Prediction2', 'Unnamed: 0.1','Unnamed: 0', 'Retweet\r'], axis = 1)

In [ ]:
df_cluster_tweet_mod.to_csv('Cluster_Tweet_Final.csv')

In [ ]:
df_hate_id = [4611,13784,24160,28866,28967,29776,
 30653,34869,37671,39597,40080,45935,47807,49567,
 49573,51222,51509,51665,56469,56667,67060,68518,79716,88796,90599,100047,101804,103635,109389,109815,112395,
 120138,120780,122624,122786,124526,149229,149229,150641,150641,150641,151279,156058]

In [ ]:
df_hate_id_string = map(str, df_hate_id)

In [ ]:
len(df_cluster_tweet_mod[df_cluster_tweet_mod.Cluster_ID.isin(df_hate_id)].groupby('Cluster_ID').count()

# Basic Features

In [ ]:
df_cluster_report = df_cluster_tweet_mod.groupby('Cluster_ID').count().reset_index()[['Cluster_ID', 'Tweet_ID']]

In [ ]:
df_cluster_report.loc[df_cluster_report.Cluster_ID == 4611] 

In [ ]:
df_cluster_report = df_cluster_report.rename(columns = {'Tweet_ID': 'Number_of_copies'})

In [ ]:
df_cluster_tweet_mod.Total_Hashtags = df_cluster_tweet_mod.Total_Hashtags.astype('int')
df_cluster_tweet_mod.Retweet_Count = df_cluster_tweet_mod.Retweet_Count.astype('int')
df_cluster_tweet_mod.Favorite_Count = df_cluster_tweet_mod.Favorite_Count.astype('int')

In [ ]:
df_cluster_report_sum = df_cluster_tweet_mod.groupby('Cluster_ID').sum().reset_index()

In [ ]:
df_cluster_report = pd.merge(df_cluster_report,df_cluster_report_sum )

In [ ]:
df_cluster_tweet_mod.Tweet_Time = pd.to_datetime(df_cluster_tweet_mod.Tweet_Time, format='%Y-%m-%d')

In [ ]:
df_cluster_tweet_mod['Tweet_Date']= df_cluster_tweet_mod.Tweet_Time.dt.date

In [ ]:
df_cluster_tweet_mod = df_cluster_tweet_mod.sort_values(by = ['Cluster_ID', 'Tweet_Date'], ascending=[True, False])

In [ ]:
df_cluster_tweet_mod_cluster_id_list = list(set(df_cluster_tweet_mod.Cluster_ID.tolist()))

In [ ]:
df_cluster_tweet_mod['Link_Count'] = df_cluster_tweet_mod['Tweet_Text'].str.count('https://')
df_cluster_tweet_mod['Hashtags_Count'] = df_cluster_tweet_mod['Tweet_Text'].str.count('#')
df_cluster_tweet_mod['Mention_Count'] = df_cluster_tweet_mod['Tweet_Text'].str.count('@')

In [ ]:
df_cluster_tweet_mod

In [ ]:
df_cluster_report

In [ ]:
import requests
g = 0
for index1,row1 in df_cluster_report.iterrows():

    df_specific = df_cluster_tweet_mod.loc[df_cluster_tweet_mod.Cluster_ID == row1.Cluster_ID]
    df_specific = df_specific.reset_index(drop = True)
    date_diff = df_specific.Tweet_Date[0] - df_specific.Tweet_Date[len(df_specific)-1]
    
    df_cluster_report.loc[index1, 'Time_Difference'] = date_diff
    df_cluster_report.loc[index1, 'Number_of_copies'] = len(df_specific)
    
    df_cluster_report.loc[index1, 'RT_Max'] = df_specific.Retweet_Count.max()
    df_cluster_report.loc[index1, 'RT_Min'] = df_specific.Retweet_Count.min()
    df_cluster_report.loc[index1, 'RT_Mean'] = df_specific.Retweet_Count.mean()
    df_cluster_report.loc[index1, 'RT_Median'] = df_specific.Retweet_Count.median()
    df_cluster_report.loc[index1, 'RT_SD'] = df_specific.Retweet_Count.std()
    
    df_cluster_report.loc[index1, 'Like_Max'] = df_specific.Favorite_Count.max()
    df_cluster_report.loc[index1, 'Like_Min'] = df_specific.Favorite_Count.min()
    df_cluster_report.loc[index1, 'Like_Mean'] = df_specific.Favorite_Count.mean()
    df_cluster_report.loc[index1, 'Like_Median'] = df_specific.Favorite_Count.median()
    df_cluster_report.loc[index1, 'Like_SD'] = df_specific.Favorite_Count.std()
    
    hashtags = []
    for index, row in df_specific.iterrows():
        hashtags.extend(re.findall(r"#\w+", row["Tweet_Text"]))
    

    total_hashtags = len(hashtags)
    unique_hashtags = len(list(set(hashtags)))
    df_cluster_report.loc[index1, 'Unique_Hashtags'] = unique_hashtags
    
    
    link = df_specific.Tweet_Text.str.extract(r"(https?://[^\s]+)")
    link_dropna = link.dropna()
    total_link = len(link_dropna)
    df_cluster_report.loc[index1, 'Total_Links'] = total_link
    


In [ ]:
df_cluster_report.Time_Difference = (df_cluster_report.Time_Difference.astype(int)/ 86400000000000).astype(int) + 1

In [ ]:
df_cluster_report

In [ ]:
len(link_dropna)

In [ ]:
df_cluster_report.columns

In [ ]:
df_alt_report.columns

In [ ]:
for index1,row1 in df_cluster_report.iterrows():
    df_specific = df_cluster_tweet_mod.loc[df_cluster_tweet_mod.Cluster_ID == row1.Cluster_ID]
    df_cluster_report.loc[index1, 'Number_of_Tweet_Lang'] = len(list(set(df_specific.Tweet_Lang.to_list())))
    df_cluster_report.loc[index1, 'Tweet_Lang'] = ','.join(list(set(df_specific.Tweet_Lang.to_list())))

In [ ]:
df_cluster_report.loc[df_cluster_report.Number_of_Tweet_Lang > 1]

In [ ]:
pd.read_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/ML Model Data/AltNews/AltNews_Report_Cluster_Model_data_Updated.csv')

In [ ]:
df_cluster_report.to_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/ML Model Data/AltNews/Cluster_Model_data.csv', index = False)

In [ ]:
df_cluster_tweet_mod.to_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/ML Model Data/AltNews/Cluster_Model_Tweet.csv', index = False)

# Sentiment Genre Perspective

In [ ]:
df_cluster_sentiment_genre = pd.read_csv('Sentiment_Genre.csv')
df_cluster_sentiment_genre = df_cluster_sentiment_genre.drop(columns = 'Unnamed: 0')
df_cluster_perspective = pd.read_csv('Perspective_API_Result_All.csv')

In [ ]:
df_cluster_sentiment_genre

In [ ]:
df_cluster_perspective_pers

In [ ]:
df_cluster_perspective_pers = df_cluster_perspective[['Cluster_ID', 'Toxic', 'Severe_Toxic','Profanity','Identity_Attack','Insult','Threat']]

In [ ]:
df_cluster_perspective_pers = df_cluster_perspective_pers.dropna()

In [ ]:
df_cluster_perspective_pers.Cluster_ID = df_cluster_perspective_pers.Cluster_ID.astype('int')

In [ ]:
df_cluster_perspective_pers= df_cluster_perspective_pers[df_cluster_perspective_pers['Toxic'] >= 0]

In [ ]:
df_cluster_sentiment_genre

In [ ]:
df_cluster_text_based = pd.merge(df_cluster_sentiment_genre, df_cluster_perspective_pers, on = 'Cluster_ID')

In [ ]:
df_cluster_text_based 

In [ ]:
df_cluster_report = pd.merge(df_cluster_report, df_cluster_text_based, on = 'Cluster_ID')

In [ ]:
df_cluster_report.to_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/ML Model Data/AltNews/Cluster_Model_data.csv', index = False)

In [ ]:
df_alt_report.columns

In [ ]:
df_cluster_tweet_mod

In [ ]:
df_cluster_report

In [ ]:
df_cluster_tweet_mod.Tweet_Time = pd.to_datetime(df_cluster_tweet_mod.Tweet_Time)

In [ ]:
df_cluster_tweet_mod['Tweet_Date_Only'] = df_cluster_tweet_mod.Tweet_Time.dt.date

In [ ]:
df_cluster_tweet_mod['Tweet_Month_Only'] = pd.to_datetime(df_cluster_tweet_mod['Tweet_Date_Only']).dt.strftime('%m-%Y')

In [ ]:
df_account = pd.read_csv('All_Fake_without_debunkers_Updated.csv')

In [ ]:
df_cluster_tweet_mod.User_ID = df_cluster_tweet_mod.User_ID.astype('int')

In [ ]:
df_cluster_tweet_mod2 = pd.merge(df_cluster_tweet_mod, df_account[['User_ID', 'verification']], how = 'left')

In [ ]:
set(df_cluster_tweet_mod2.verification.tolist())

In [ ]:
def loc_soc(df):
    df = df.assign(is_verified=0)
    df.loc[df['verification'] == 'Verified', 'is_verified'] = 1
    return df
df_cluster_tweet_mod2 = loc_soc(df_cluster_tweet_mod2)

In [ ]:
df_cluster_tweet_mod2.Tweet_Date_Only = pd.to_datetime(df_cluster_tweet_mod2['Tweet_Date_Only'])

In [ ]:
for index1,row1 in df_cluster_report.iterrows():
    df_specific = df_cluster_tweet_mod2.loc[df_cluster_tweet_mod2.Cluster_ID == row1.Cluster_ID]
    df_specific = df_specific.sort_values(by = 'Tweet_Time', ascending = False)
    
    weekly_spike_count = (df_specific.groupby('Cluster_ID').resample('W', on='Tweet_Date_Only', origin= 'start').count())
    bi_weekly_spike_count = (df_specific.groupby('Cluster_ID').resample('2W', on='Tweet_Date_Only', origin= 'start').count())
    monthly_spike_count = (df_specific.groupby('Cluster_ID').resample('M', on='Tweet_Date_Only', origin= 'start').count())
    
    weekly_spike_sum = (df_specific.groupby('Cluster_ID').resample('W', on='Tweet_Date_Only', origin= 'start').sum())
    bi_weekly_spike_sum = (df_specific.groupby('Cluster_ID').resample('2W', on='Tweet_Date_Only', origin= 'start').sum())
    monthly_spike_sum = (df_specific.groupby('Cluster_ID').resample('M', on='Tweet_Date_Only', origin= 'start').sum())
    
    
    weekly_spike = weekly_spike_count.loc[weekly_spike_count.Tweet_ID > 0]
    bi_weekly_spike = bi_weekly_spike_count.loc[bi_weekly_spike_count.Tweet_ID > 0]
    monthly_spike = monthly_spike_count.loc[monthly_spike_count.Tweet_ID > 0]
    
    weekly_ver_spike = weekly_spike_sum.loc[weekly_spike_sum.is_verified > 0]
    bi_weekly_ver_spike = bi_weekly_spike_sum.loc[bi_weekly_spike_sum.is_verified > 0]
    monthly_ver_spike = monthly_spike_sum.loc[monthly_spike_sum.is_verified > 0]
    
    df_cluster_report.loc[index1, 'Total_Weekly_Spike'] = len(weekly_spike_count)
    df_cluster_report.loc[index1, 'Total_BiWeekly_Spike'] = len(bi_weekly_spike_count)
    df_cluster_report.loc[index1, 'Total_Monthly_Spike'] = len(monthly_spike_count)
    
    df_cluster_report.loc[index1, 'Weekly_Largest_Spike'] = weekly_spike.Tweet_ID.max()
    df_cluster_report.loc[index1, 'BiWeekly_Largest_Spike'] = bi_weekly_spike.Tweet_ID.max()
    df_cluster_report.loc[index1, 'Monthly_Largest_Spike'] = monthly_spike.Tweet_ID.max()
    
    df_cluster_report.loc[index1, 'Weekly_Verified_Spike'] = len(weekly_ver_spike)
    df_cluster_report.loc[index1, 'BiWeekly_Verified_Spike'] = len(bi_weekly_ver_spike)
    df_cluster_report.loc[index1, 'Monthly_Verified_Spike'] = len(monthly_ver_spike)
    
    df_cluster_report.loc[index1, 'Total_Verified'] = sum(df_specific.is_verified)
    
    
    

In [ ]:
df_cluster_report

In [ ]:
df_cluster_report.to_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/ML Model Data/AltNews/Cluster_Model_data.csv', index = False)

In [ ]:
df_cluster_report

# Account Based Features

In [ ]:
df_modularity = pd.read_csv('Modularity_Class.csv')

In [ ]:
df_modularity['User_ID'] = df_modularity['Label']

In [ ]:
df_modularity[['User_ID', 'modularity_class']]

In [ ]:
df_account[['User_ID', 'acc_type', 'party','Political', 'category', 'Influencer']]

In [ ]:
df_duplicator_info = pd.merge(df_modularity[['User_ID', 'modularity_class']], df_account[['User_ID', 'acc_type', 'party','Political', 'category', 'Influencer']])

In [ ]:
df_duplicator_info

In [ ]:
df_pol_available = pd.merge(df_cluster_tweet_mod2, df_account[['User_ID', 'party','Political', 'category', 'Influencer']])

In [ ]:
df_pol_available 

In [ ]:
for index1,row1 in df_cluster_report.iterrows():
    df_specific = df_pol_available.loc[df_pol_available.Cluster_ID == row1.Cluster_ID]
    df_specific = df_specific.sort_values(by = 'Tweet_Time', ascending = False)
    
    df_specific_duplicator_info = df_duplicator_info[df_duplicator_info.User_ID.isin(df_specific.User_ID)]
    df_merged_dup_info =  pd.merge(df_specific, df_specific_duplicator_info, on = ['User_ID', 'party'])
    
    df_specific_pol = df_specific.loc[df_specific.Political == 'Political']
    df_specific_bjp = df_specific.loc[df_specific.party == 'BJP']
    df_specific_inc = df_specific.loc[df_specific.party == 'INC']
    df_specific_influ = df_specific.loc[df_specific.Influencer == 'Influencer']
    
    df_cluster_report.loc[index1, 'Total_Political_Accounts'] = len(df_specific_pol)
    df_cluster_report.loc[index1,'Total_BJP_Accounts'] = len(df_specific_bjp)
    df_cluster_report.loc[index1,'Total_INC_Accounts'] = len(df_specific_inc)
    df_cluster_report.loc[index1,'Total_Influencer_Accounts'] = len(df_specific_inc)
    
    number_of_duplicator_clusters = len(df_specific_duplicator_info.groupby('modularity_class').count())
    number_of_duplicator_accounts = len(df_specific_duplicator_info)
    
    duplicator_clusters_max = df_specific_duplicator_info.groupby('modularity_class').count().max().User_ID
    duplicator_clusters_with_verified = len(df_specific_duplicator_info.loc[df_specific_duplicator_info.acc_type == 'Verified'].groupby('modularity_class').count())
    duplicator_clusters_with_political = len(df_specific_duplicator_info.loc[df_specific_duplicator_info.Political == 'Political'].groupby('modularity_class').count())
    duplicator_clusters_with_bjp = len(df_specific_duplicator_info.loc[df_specific_duplicator_info.party == 'BJP'].groupby('modularity_class').count())
    duplicator_clusters_with_inc = len(df_specific_duplicator_info.loc[df_specific_duplicator_info.party == 'INC'].groupby('modularity_class').count())

    duplicator_clusters_with_bjp_count = len(df_merged_dup_info.loc[df_merged_dup_info.party == 'BJP'])
    duplicator_clusters_with_inc_count = len(df_merged_dup_info.loc[df_merged_dup_info.party == 'INC'])
    
    df_cluster_report.loc[index1, 'Number_of_Duplicator_Account_Clusters'] = number_of_duplicator_clusters
    df_cluster_report.loc[index1,'Number_of_Duplicator_Accounts'] = number_of_duplicator_accounts
    df_cluster_report.loc[index1,'Number_of_Duplicator_Accounts_with_BJP'] = duplicator_clusters_with_bjp_count
    df_cluster_report.loc[index1,'Number_of_Duplicator_Accounts_with_INC'] = duplicator_clusters_with_inc_count
    
    df_cluster_report.loc[index1,'Maximum_Duplicator_Account_Cluster_Size'] = duplicator_clusters_max
    df_cluster_report.loc[index1,'Number_of_Duplicator_Account_Clusters_with_Verified'] = duplicator_clusters_with_verified
    df_cluster_report.loc[index1,'Number_of_Duplicator_Account_Clusters_with_Political'] = duplicator_clusters_with_political
    df_cluster_report.loc[index1,'Number_of_Duplicator_Account_Clusters_with_BJP'] = duplicator_clusters_with_bjp
    df_cluster_report.loc[index1,'Number_of_Duplicator_Account_Clusters_with_INC'] = duplicator_clusters_with_inc
    
    
    

In [ ]:
df_cluster_report

In [ ]:
df_cluster_report.to_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/ML Model Data/AltNews/Cluster_Model_data.csv', index = False)

In [ ]:
df_cluster_report.columns

In [ ]:
df_cluster_report.loc[(df_cluster_report.label == 'positive') &
                     (df_cluster_report.score > 0.7) &
                     (df_cluster_report.Number_of_copies  >= 10)]

In [ ]:
df_cluster_report

In [ ]:
df_cluster_tweet_mod2['Text_without_Link'] = df_cluster_tweet_mod2['Tweet_Text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
df_cluster_tweet_mod2['Text_without_Link']=df_cluster_tweet_mod2['Text_without_Link'].str.replace('(@\w+.*?)',"")
df_cluster_tweet_mod2['Link_Count'] = df_cluster_tweet_mod2['Tweet_Text'].str.count('https://')
df_cluster_tweet_mod2['Hashtags_Count'] = df_cluster_tweet_mod2['Tweet_Text'].str.count('#')
df_cluster_tweet_mod2['Mention_Count'] = df_cluster_tweet_mod2['Tweet_Text'].str.count('@')
df_cluster_tweet_mod2['Count_Char']=df_cluster_tweet_mod2['Text_without_Link'].str.len()
df_cluster_tweet_mod2['Count_Word']= df_cluster_tweet_mod2['Text_without_Link'].str.split().str.len()

In [ ]:
df_cluster_tweet_mod2

In [ ]:
for index1,row1 in df_cluster_report.iterrows():
    df_specific = df_cluster_tweet_mod2.loc[df_cluster_tweet_mod2.Cluster_ID == row1.Cluster_ID]
    df_specific = df_specific.sort_values(by = 'Tweet_Time', ascending = False)
    
    df_cluster_report.loc[index1,'Mean_Character'] = df_specific.Count_Char.mean()
    df_cluster_report.loc[index1,'Mean_Word'] = df_specific.Count_Word.mean()
    df_cluster_report.loc[index1,'Median_Character'] = df_specific.Count_Char.median()
    df_cluster_report.loc[index1,'Median_Word'] = df_specific.Count_Word.median()
    df_cluster_report.loc[index1,'Total_Link'] = df_specific.Link_Count.sum()
    df_cluster_report.loc[index1,'Total_Hashtag'] = df_specific.Hashtags_Count.sum()
    df_cluster_report.loc[index1,'Total_Mention'] = df_specific.Mention_Count.sum()

In [ ]:
df_cluster_report.to_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/ML Model Data/AltNews/Cluster_Model_data.csv', index = False)